In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit
from pyspark.sql.functions import col, regexp_replace, trim, when, regexp_extract
from pyspark.sql.types import *
from pyspark.sql.functions import col, isnan, when, count ,date_format,to_date,to_timestamp

In [2]:
# Create Spark Session
spark = SparkSession.builder \
    .appName("DataProcessing") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/01/14 13:22:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark 

In [4]:
#Customers Schema
customers_schema = StructType([
    StructField("customer_id", StringType(), True),
    StructField("customer_name", StringType(), True),
    StructField("city", StringType(), True),
])  

In [6]:
# Reading Customer CSV
customers_df = spark.read \
    .format("csv") \
    .option("header", True) \
    .schema(customers_schema) \
    .load("../data/customers.csv")

In [7]:
# Check schemas
print("Customers DataFrame Schema:")
customers_df.printSchema()

Customers DataFrame Schema:
root
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- city: string (nullable = true)



###  Checkin the number of rows in the data

In [8]:
num_rows = customers_df.count()
print(f"Number of rows: {num_rows}")


Number of rows: 200


### Check for missing values

In [64]:
# Counting missing values for each column
missing_values = customers_df.select(
    [count(when(col(c).isNull(), c)).alias(c) for c in customers_df.columns]
)
missing_values.show()

+-----------+-------------+----+
|customer_id|customer_name|city|
+-----------+-------------+----+
|          0|           22|  21|
+-----------+-------------+----+



### Substitute with a Default Value 

For the missing values in Customers , default values will be used eg Unknown 

In [65]:
from pyspark.sql import functions as F

def clean_customer_names(customers_df):
    """
    Efficiently handle missing names in customer dataset using PySpark.
    Using na.fill() is more performant than withColumn() for simple replacements.
    
    Args:
        customers_df: PySpark DataFrame containing customer data
        
    Returns:
        PySpark DataFrame with missing names replaced with "Unknown"
    """
    # Get list of name columns (assuming they might be first_name, last_name, or name)
    name_columns = [col for col in customers_df.columns 
                   if any(name_field in col.lower() 
                         for name_field in ['customer_name','city'])]
    
    # Create dictionary of columns to fill
    fill_dict = {col: "Unknown" for col in name_columns}
    
    # Use na.fill() which is more efficient than withColumn() for simple replacements
    cleaned_df = customers_df.na.fill(fill_dict)
    
    # Cache the result if you'll be using it multiple times
    cleaned_df = cleaned_df.cache()  # Uncomment if needed
    
    return cleaned_df


# Example usage
cleaned_customers_df = clean_customer_names(customers_df)


25/01/14 15:13:01 WARN CacheManager: Asked to cache already cached data.


In [66]:
# Counting missing values for each column
missing_values = cleaned_customers_df.select(
    [count(when(col(c).isNull(), c)).alias(c) for c in cleaned_customers_df.columns]
)
missing_values.show()

+-----------+-------------+----+
|customer_id|customer_name|city|
+-----------+-------------+----+
|          0|            0|   0|
+-----------+-------------+----+



### Standardizing  Data Types

In [33]:
from pyspark.sql import functions as F
from pyspark.sql.types import LongType
import traceback

def standardize_customer_id(df, column_name: str = "customer_id"):
    ''' Standardizes the customer_id column by converting valid numeric strings to LongType 
    and invalid values to None. The function also optimizes memory usage, minimizes shuffle 
    operations, and adapts partitioning based on data size.'''
    try:
        if df is None or column_name not in df.columns:
            print("Invalid DataFrame or column name.")
            return None
        
        # Regex pattern for numeric values
        pattern = "^[0-9]+$"
        
        # Apply transformation
        standardized_df = df.withColumn(
            column_name,
            F.when(
                F.col(column_name).rlike(pattern),
                F.col(column_name).cast(LongType())
            ).otherwise(None)
        )
        
        # Log row count
        row_count = standardized_df.count()
        print(f"Row count after transformation: {row_count}")
        
        # Repartition if needed (optional for testing)
        standardized_df = standardized_df.repartition(200)
        
        return standardized_df
    except Exception as e:
        print(f"Error in function: {str(e)}")
        traceback.print_exc()
        return None


In [34]:
standardized_customers_df = standardize_customer_id(customers_df, column_name="customer_id")


Row count after transformation: 200


In [35]:
#Verifying the schema
standardized_customers_df.printSchema()

root
 |-- customer_id: long (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- city: string (nullable = true)



In [27]:
print(type(customers_df))
customers_df.printSchema()


<class 'pyspark.sql.dataframe.DataFrame'>
root
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- city: string (nullable = true)



In [39]:
from pyspark.sql import functions as F
from pyspark.sql.types import LongType, IntegerType
import traceback

def filter_invalid_customer_id(df, column_name: str = "customer_id", invalid_values: list = [-99999, 1e6]):
    """
    Filters out rows where the customer_id column has nonsensical values like -99999 or 1e6.
    This function uses efficient filtering methods optimized for computational cost and memory usage.

    Args:
        df (pyspark.sql.DataFrame): The input PySpark DataFrame containing the customer_id column.
        column_name (str, optional): The name of the column to check for invalid values (default is "customer_id").
        invalid_values (list, optional): List of values that should be considered invalid and filtered out (default is [-99999, 1e6]).

    Returns:
        pyspark.sql.DataFrame:
            A new DataFrame with rows that do not contain invalid customer_id values. 
            Returns None if the input DataFrame is invalid or an error occurs.

    Example:
        >>> valid_customers_df = filter_invalid_customer_id(customers_df, column_name="customer_id")
        >>> valid_customers_df.show()
    """
    try:
        if df is None or column_name not in df.columns:
            print("Invalid DataFrame or column name.")
            return None
        
        # Convert invalid_values list to a set for faster lookup
        invalid_values_set = set(invalid_values)
        
        # Create the filter condition to exclude rows with invalid customer_id values
        filter_condition = ~F.col(column_name).isin(invalid_values_set)
        
        # Apply the filter and return the DataFrame with valid customer_id values
        df_filtered = df.filter(filter_condition)

        return df_filtered
    except Exception as e:
        print(f"Error in function: {str(e)}")
        traceback.print_exc()
        return None


In [40]:
# Assuming customers_df is your original DataFrame
valid_customers_df = filter_invalid_customer_id(customers_df, column_name="customer_id", invalid_values=[-99999, 1e6])

# Verifying the result
valid_customers_df.show()


+-----------+-----------------+-----------------+
|customer_id|    customer_name|             city|
+-----------+-----------------+-----------------+
|     789221|      Info Stores|     Polokwane 11|
|     789301|             null|           durban|
|     789121|      Coolblue 96|        Cape Town|
|     789501|     Logic Stores|port elizabeth-10|
|     789201|         coolblue|   port elizabeth|
|     789422|    viveks stores|             null|
|     789603|        Rel Fresh|      East London|
|     789201|        rel fresh|             null|
|     789303|             null|INVALID_CITY20499|
|     123.45|             null|INVALID_CITY55754|
|     789520|       Lotus Mart|             null|
|     789902|    Viveks Stores|      East London|
|     789522|    Elite Mart_74|INVALID_CITY58916|
|     789101|    Viveks Stores|          durban1|
|         ID|     elite mart98|         Pretoria|
|     789301|    viveks stores|     East London1|
|     789420|Acclaimed Stores7|        Polokwane|


In [41]:
from pyspark.sql import functions as F
import traceback

def show_invalid_customer_ids(df, column_name: str = "customer_id", invalid_values: list = [-99999, 1e6]):
    """
    Displays all rows where customer_id has nonsensical values such as -99999 or 1e6.
    
    This function identifies rows with invalid customer_id values for inspection without making any changes to the data.

    Args:
        df (pyspark.sql.DataFrame): The input PySpark DataFrame containing the customer_id column.
        column_name (str, optional): The name of the column to check for invalid values (default is "customer_id").
        invalid_values (list, optional): List of invalid customer_id values (default is [-99999, 1e6]).

    Returns:
        pyspark.sql.DataFrame:
            A DataFrame containing the rows with invalid customer_id values.
            Returns None if the input DataFrame is invalid or an error occurs.

    Example:
        >>> invalid_customer_ids_df = show_invalid_customer_ids(customers_df)
        >>> invalid_customer_ids_df.show()
    """
    try:
        if df is None or column_name not in df.columns:
            print("Invalid DataFrame or column name.")
            return None
        
        # Convert invalid values list to a set for efficient lookup
        invalid_values_set = set(invalid_values)
        
        # Create the filter condition for invalid customer_ids
        filter_condition = F.col(column_name).isin(invalid_values_set)
        
        # Apply the filter and return rows that match the condition
        df_invalid = df.filter(filter_condition)

        return df_invalid
    except Exception as e:
        print(f"Error in function: {str(e)}")
        traceback.print_exc()
        return None


In [42]:
# Show rows with invalid customer_id values first
invalid_customer_ids_df = show_invalid_customer_ids(customers_df)

# Verifying the invalid rows
invalid_customer_ids_df.show()

# Once verified, drop rows with invalid customer_id values
cleaned_customers_df = drop_invalid_customer_ids(customers_df)

# Verifying the result after dropping invalid rows
bcleaned_customers_df.show()


+-----------+--------------------+-----------------+
|customer_id|       customer_name|             city|
+-----------+--------------------+-----------------+
|  1000000.0|                null|        polokwane|
|     -99999|         PROPEL MART|        Cape Town|
|  1000000.0|                null|        Polokwane|
|     -99999|        Sorefoz Mart|      pretoria 14|
|  1000000.0|          Elite Mart|      East London|
|  1000000.0|        logic stores|        kimberley|
|     -99999|      propel mart_11|             null|
|     -99999|         Expert Mart|        nelspruit|
|     -99999|   Chiptec Stores_42|      cape town 9|
|     -99999|       lotus mart 74|        Kimberley|
|  1000000.0|        logic stores|        Nelspruit|
|  1000000.0|            COOLBLUE|           durban|
|     -99999|expression stores_62|        Polokwane|
|  1000000.0|          ELITE MART|     Nelspruit 18|
|     -99999|            COOLBLUE|INVALID_CITY62618|
|     -99999|          Lotus Mart|        Cape

NameError: name 'drop_invalid_customer_ids' is not defined

In [46]:
## Hashing


In [45]:
from pyspark.sql.functions import md5, concat, col, lit, coalesce
from pyspark.sql.types import StringType

# 1. Optimize by selecting only required columns
customers_df_opt = customers_df.select("customer_name", "city")

# 2. Create MD5 hash using concatenated values
customers_with_hash = customers_df_opt.withColumn(
    "customer_id_hash",
    md5(concat(
        coalesce(col("customer_name"), lit("")),  # Handle NULL values
        lit("_"),  # Delimiter for better uniqueness
        coalesce(col("city"), lit(""))
    ))
)

# 3. Cache the result if you'll be using it multiple times
# 3. Cache the result if you'll be using it multiple times
customers_with_hash.cache()

# 4. Perform any additional operations with the hashed data
# For example, select final columns needed
final_df = customers_with_hash.select(
    "customer_id_hash",
    "customer_name",
    "city"


# 5. Write the results (if needed)
# Use appropriate partitioning based on your data size
final_df.write \
    .mode("overwrite") \
    .partitionBy("city") \
    .format("parquet") \
    .save("output_path")

# 6. To verify the results (optional)
print("Number of records processed:", final_df.count())
print("\nSchema of final dataframe:")
final_df.printSchema()

# 7. Show a sample of the results (optional)
print("\nSample of processed data:")
final_df.show(5, truncate=False)

Number of records processed: 200

Schema of final dataframe:
root
 |-- customer_id_hash: string (nullable = false)
 |-- customer_name: string (nullable = true)
 |-- city: string (nullable = true)


Sample of processed data:
+--------------------------------+-------------+-----------------+
|customer_id_hash                |customer_name|city             |
+--------------------------------+-------------+-----------------+
|1e7746579353fc48ce01dad8221156c9|Info Stores  |Polokwane 11     |
|c270645522c203f91626a17998317cc8|null         |durban           |
|b979555b640767d3f2603e29319024f0|Coolblue 96  |Cape Town        |
|6d9be2294eb49598988c36e46e3d630a|Logic Stores |port elizabeth-10|
|4ac660ea4ac050c24e503263f4e29107|coolblue     |port elizabeth   |
+--------------------------------+-------------+-----------------+
only showing top 5 rows



In [47]:
###remove numbers

In [56]:
from pyspark.sql.functions import regexp_replace, initcap, when, col, lower

# Reference cities list
cities = [
    "Johannesburg", "Cape Town", "Durban", "Pretoria", "Port Elizabeth", 
    "East London", "Bloemfontein", "Nelspruit", "Polokwane", "Kimberley"
]

# Create a broadcast variable for efficient lookup
cities_broadcast = spark.sparkContext.broadcast([city.lower() for city in cities])

# Clean and standardize cities in one pass
cleaned_customers_df = customers_df.withColumn(
    "city",
    when(
        lower(regexp_replace(
            regexp_replace(col("city"), r"[-_]\d+$|[\s]+\d+$", ""),  # Remove numeric noise
            r"\s+", " "  # Standardize spaces
        )).isin(cities_broadcast.value),
        initcap(regexp_replace(col("city"), r"[-_]\d+$|[\s]+\d+$", ""))
    ).otherwise(col("city"))
)

# Show results
cleaned_customers_df.select("customer_id", "customer_name", "city").show()

+-----------+-----------------+-----------------+
|customer_id|    customer_name|             city|
+-----------+-----------------+-----------------+
|     789221|      Info Stores|        Polokwane|
|     789301|             null|           Durban|
|     789121|      Coolblue 96|        Cape Town|
|     789501|     Logic Stores|   Port Elizabeth|
|     789201|         coolblue|   Port Elizabeth|
|     789422|    viveks stores|             null|
|     789603|        Rel Fresh|      East London|
|  1000000.0|             null|        Polokwane|
|     789201|        rel fresh|             null|
|     789303|             null|INVALID_CITY20499|
|     123.45|             null|INVALID_CITY55754|
|     789520|       Lotus Mart|             null|
|     789902|    Viveks Stores|      East London|
|     789522|    Elite Mart_74|INVALID_CITY58916|
|     789101|    Viveks Stores|          durban1|
|         ID|     elite mart98|         Pretoria|
|     789301|    viveks stores|     East London1|


In [57]:
#Removing null values and Invali_city

In [60]:
from pyspark.sql.functions import regexp_replace, initcap, when, col, lower, lit

# Reference cities list
cities = [
    "Johannesburg", "Cape Town", "Durban", "Pretoria", "Port Elizabeth", 
    "East London", "Bloemfontein", "Nelspruit", "Polokwane", "Kimberley"
]

# Create a broadcast variable for efficient lookup
cities_broadcast = spark.sparkContext.broadcast([city.lower() for city in cities])

# Clean and standardize cities, handle null names and invalid cities
cleaned_customers_df = customers_df.withColumn(
   "customer_name",
    when(col("customer_name").isNull(), lit("unknown"))
    .otherwise(col("customer_name"))
).withColumn(
    "city",
    when(
        (lower(col("city")).like("INVALID_CITY%")) |  # Match INVALID_CITY pattern
        (col("city").isNull()),                       # Handle null cities
        lit("unknown")
    ).otherwise(
        when(
            lower(regexp_replace(
                regexp_replace(col("city"), r"[-_]\d+$|[\s]+\d+$", ""),
                r"\s+", " "
            )).isin(cities_broadcast.value),
            initcap(regexp_replace(col("city"), r"[-_]\d+$|[\s]+\d+$", ""))
        ).otherwise(col("city"))
    )
)

# Show results
cleaned_customers_df.select("customer_name", "customer_id", "city").show()

+-----------------+-----------+-----------------+
|    customer_name|customer_id|             city|
+-----------------+-----------+-----------------+
|      Info Stores|     789221|        Polokwane|
|          unknown|     789301|           Durban|
|      Coolblue 96|     789121|        Cape Town|
|     Logic Stores|     789501|   Port Elizabeth|
|         coolblue|     789201|   Port Elizabeth|
|    viveks stores|     789422|          unknown|
|        Rel Fresh|     789603|      East London|
|          unknown|  1000000.0|        Polokwane|
|        rel fresh|     789201|          unknown|
|          unknown|     789303|INVALID_CITY20499|
|          unknown|     123.45|INVALID_CITY55754|
|       Lotus Mart|     789520|          unknown|
|    Viveks Stores|     789902|      East London|
|    Elite Mart_74|     789522|INVALID_CITY58916|
|    Viveks Stores|     789101|          durban1|
|     elite mart98|         ID|         Pretoria|
|    viveks stores|     789301|     East London1|


In [61]:
from pyspark.sql.functions import regexp_replace, initcap, when, col, lower, lit, upper

# Reference cities list
cities = [
    "Johannesburg", "Cape Town", "Durban", "Pretoria", "Port Elizabeth", 
    "East London", "Bloemfontein", "Nelspruit", "Polokwane", "Kimberley"
]

# Create a broadcast variable for efficient lookup
cities_broadcast = spark.sparkContext.broadcast([city.lower() for city in cities])

# Clean and standardize cities, handle null names and invalid cities
cleaned_customers_df = customers_df.withColumn(
    "customer_name",
    when(col("customer_name").isNull(), lit("Unknown"))
    .otherwise(col("customer_name"))
).withColumn(
    "city",
    when(
        (upper(col("city")).rlike("INVALID_CITY[0-9]*")) |  # Match any case of INVALID_CITY followed by numbers
        (col("city").isNull()),
        lit("Unknown")
    ).otherwise(
        when(
            lower(regexp_replace(
                regexp_replace(col("city"), r"[-_]\d+$|[\s]+\d+$", ""),
                r"\s+", " "
            )).isin(cities_broadcast.value),
            initcap(regexp_replace(col("city"), r"[-_]\d+$|[\s]+\d+$", ""))
        ).otherwise(col("city"))
    )
)

# Show results
cleaned_customers_df.select( "customer_id","customer_name","city").show(truncate=False)

# Optional: Check if any INVALID_CITY patterns remain
remaining_invalid = cleaned_customers_df.filter(upper(col("city")).rlike("INVALID_CITY"))
print("Remaining invalid cities count:", remaining_invalid.count())

+-----------+-----------------+--------------+
|customer_id|customer_name    |city          |
+-----------+-----------------+--------------+
|789221     |Info Stores      |Polokwane     |
|789301     |Unknown          |Durban        |
|789121     |Coolblue 96      |Cape Town     |
|789501     |Logic Stores     |Port Elizabeth|
|789201     |coolblue         |Port Elizabeth|
|789422     |viveks stores    |Unknown       |
|789603     |Rel Fresh        |East London   |
|1000000.0  |Unknown          |Polokwane     |
|789201     |rel fresh        |Unknown       |
|789303     |Unknown          |Unknown       |
|123.45     |Unknown          |Unknown       |
|789520     |Lotus Mart       |Unknown       |
|789902     |Viveks Stores    |East London   |
|789522     |Elite Mart_74    |Unknown       |
|789101     |Viveks Stores    |durban1       |
|ID         |elite mart98     |Pretoria      |
|789301     |viveks stores    |East London1  |
|789420     |Acclaimed Stores7|Polokwane     |
|789420     |

In [62]:
#cHECKING FOR DUPLICATES 

In [63]:
from pyspark.sql.functions import regexp_replace, initcap, when, col, lower, lit, upper, count

# Your existing cleaning code remains the same up to the cleaned_customers_df creation

# First, let's check for duplicates
print("\nChecking for duplicates...")
duplicate_check = cleaned_customers_df.groupBy("customer_id", "customer_name", "city") \
    .agg(count("*").alias("count")) \
    .filter(col("count") > 1)

print("Number of duplicate records found:", duplicate_check.count())

# Show duplicate records if any exist
if duplicate_check.count() > 0:
    print("\nDuplicate records found:")
    duplicate_check.show(truncate=False)

# Remove duplicates
deduped_customers_df = cleaned_customers_df.dropDuplicates(["customer_id", "customer_name", "city"])

print("\nOriginal record count:", cleaned_customers_df.count())
print("Record count after removing duplicates:", deduped_customers_df.count())

# Show final results
print("\nFinal cleaned and deduplicated data:")
deduped_customers_df.select("customer_id", "customer_name", "city").show(truncate=False)

# Save the final cleaned and deduplicated dataframe
deduped_customers_df.createOrReplaceTempView("cleaned_customers")


Checking for duplicates...
Number of duplicate records found: 4

Duplicate records found:
+-----------+-------------+---------+-----+
|customer_id|customer_name|city     |count|
+-----------+-------------+---------+-----+
|1000000.0  |Unknown      |Polokwane|2    |
|123.45     |Unknown      |Unknown  |3    |
|789303     |Unknown      |Unknown  |2    |
|789122     |logic stores |Cape Town|2    |
+-----------+-------------+---------+-----+


Original record count: 200
Record count after removing duplicates: 195

Final cleaned and deduplicated data:
+-----------+----------------+--------------+
|customer_id|customer_name   |city          |
+-----------+----------------+--------------+
|1000000.0  |Unknown         |Polokwane     |
|789420     |Acclaimed Stores|Unknown       |
|abcd       |Propel Mart     |Nelspruit     |
|789320     |Propel Mart_95  |Unknown       |
|123.45     |Rel Fresh       |East London   |
|789201     |Sorefoz Mart    |Durban        |
|789102     |Lotus Mart      |Ne